<a href="https://colab.research.google.com/github/dhdmsdud/TIL/blob/master/%EC%B6%94%EC%B2%9C_%EC%8B%9C%EC%8A%A4%ED%85%9C(Recommender_System).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 추천 시스템 (Recommender Systems)

* 컨텐츠 기반 필터링 (content-based filtering)

  --> 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
* 협업 필터링 (collaborative filtering)

  --> 사용자와 항목간의 유사성을 동시에 사용해 추천
* 두가지를 조합한 hybrid 방식도 가능

### Surprise

* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* scikit-learn과 유사한 사용 방법

In [2]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8MB 211kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617544 sha256=48b1325db8c5705d15cc2154173c13b3eee02dc32e8900ffb3c9a610ff6cea5f
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


간단한 surprise 실습

In [3]:
from surprise import SVD, Dataset
from surprise.model_selection import cross_validate
import numpy as np
import pandas as pd

In [4]:
# data 10개만 추출
data = Dataset.load_builtin('ml-100k', prompt=False)
data.raw_ratings[:10]

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [5]:
model = SVD()

In [6]:
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9356  0.9333  0.9337  0.9415  0.9309  0.9350  0.0036  
MAE (testset)     0.7367  0.7361  0.7360  0.7416  0.7319  0.7365  0.0031  
Fit time          3.95    3.95    3.95    3.94    3.83    3.93    0.05    
Test time         0.13    0.19    0.18    0.14    0.25    0.18    0.04    


{'fit_time': (3.9543559551239014,
  3.9548027515411377,
  3.953601598739624,
  3.93623685836792,
  3.828615427017212),
 'test_mae': array([0.73674646, 0.73607452, 0.7360346 , 0.74162229, 0.73185411]),
 'test_rmse': array([0.9355718 , 0.93332583, 0.93374201, 0.94152989, 0.93094796]),
 'test_time': (0.12703466415405273,
  0.1912229061126709,
  0.18218374252319336,
  0.13966774940490723,
  0.2500741481781006)}

### 컨텐츠 기반 필터링 (Content-based Filtering)

* 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
* ex) 내가 지금 까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
* 유사도를 기반으로 추천
  * 장점
    * 많은 수의 사용자를 대상으로 쉽게 확장 가능
    * 사용자가 관심을 갖지 않던 상품 추천 가능
  * 단점
    * 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
    * 사용자의 기존 관심사항을 기반으로만 추천 가능

* 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기
* 나와 가장 높은 유사도를 가진 사용자의 시청 목록을 추천

In [7]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)

In [8]:
# 0부터 시작하도록
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [12]:
# 인접행렬의 크기
n_users  = np.max(raw_data[:, 0]) # 총 user 수
n_movies = np.max(raw_data[:, 1]) # 총 movie 수
shape    = (n_users + 1, n_movies + 1) # 0부터 시작했으니 +1을 주는건가..?
shape

(943, 1682)

In [13]:
# 인접행렬 만들기 --> 1이 있는 위치에 데이터(id, movie)가 있음
adj_matrix = np.ndarray(shape, dtype=int)

for user_id, movie_id, rating, time in raw_data :
    adj_matrix[user_id][movie_id] = 1

adj_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [18]:
my_id, my_vector = 0, adj_matrix[0] # my_vector는 인접행렬의 0번째
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix) :
    if my_id != user_id : # 다른 id끼리 비교
        similarity = np.dot(my_vector, user_vector) # 유사성 비교
        if similarity > best_match : # 유사성이 더 크면 교체
            best_match        = similarity
            best_match_id     = user_id
            best_match_vector = user_vector

print('Best_match : {}, Best_match_id : {}'.format(best_match, best_match_id))

Best_match : 183, Best_match_id : 275


In [15]:
# 추천리스트
recommend_list = []

for i, log in enumerate(zip(my_vector, best_match_vector)) :
    log1, log2 = log
    if log1 < 1 and log2 > 0 : 
        recommend_list.append(i)
    
print(recommend_list)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

* 유클리드 거리를 사용해 추천
$$euclidean = \sqrt{\sum_{d=1}^{D}(A_i - B_i)^2}$$
* 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [19]:
my_id, my_vector = 0, adj_matrix[0] # my_vector는 인접행렬의 0번째
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix) :
    if my_id != user_id : # 다른 id끼리 비교
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector))) # 유클리드 공식 풀이
        if euclidean_dist < best_match : # 유클리드가 더 작으면 교체
            best_match        = euclidean_dist
            best_match_id     = user_id
            best_match_vector = user_vector

print('Best_match : {}, Best_match_id : {}'.format(best_match, best_match_id))

Best_match : 14.832396974191326, Best_match_id : 737


In [20]:
# 추천리스트
recommend_list = []

for i, log in enumerate(zip(my_vector, best_match_vector)) :
    log1, log2 = log
    if log1 < 1 and log2 > 0 : 
        recommend_list.append(i)
    
print(recommend_list)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


* 코사인 유사도를 사용해 추천

\begin{equation}
cos \theta = \frac{A \cdot B}{||A|| \times ||B||}
\end{equation}
* 두 벡터가 이루고 있는 각을 계산

In [29]:
# 코사인 공식 풀이를 함수로
def compute_cos_similarity(v1, v2) :
    norm1 = np.sqrt(np.sum(np.square(v1))) # 분모
    norm2 = np.sqrt(np.sum(np.square(v2)))
    dot = np.dot(v1, v2) # 분자

    return dot / (norm1 * norm2)

In [30]:
my_id, my_vector = 0, adj_matrix[0] # my_vector는 인접행렬의 0번째
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix) :
    if my_id != user_id : # 다른 id끼리 비교
        cos_similarity = compute_cos_similarity(my_vector, user_vector) # 코사인 공식 함수 적용
        if cos_similarity > best_match : # 코사인 유사도가 더 크면 교체
            best_match        = cos_similarity
            best_match_id     = user_id
            best_match_vector = user_vector

print('Best_match : {}, Best_match_id : {}'.format(best_match, best_match_id))

Best_match : 0.5278586163659506, Best_match_id : 915


In [31]:
# 추천리스트
recommend_list = []

for i, log in enumerate(zip(my_vector, best_match_vector)) :
    log1, log2 = log
    if log1 < 1 and log2 > 0 : 
        recommend_list.append(i)
    
print(recommend_list)

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [32]:
adj_matrix = np.ndarray(shape, dtype=int)

for user_id, movie_id, rating, time in raw_data :
    adj_matrix[user_id][movie_id] = rating

adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [33]:
# 유클리드 적용
my_id, my_vector = 0, adj_matrix[0] # my_vector는 인접행렬의 0번째
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix) :
    if my_id != user_id : # 다른 id끼리 비교
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector))) # 유클리드 공식 풀이
        if euclidean_dist < best_match : # 유클리드가 더 작으면 교체
            best_match        = euclidean_dist
            best_match_id     = user_id
            best_match_vector = user_vector

print('Best_match : {}, Best_match_id : {}'.format(best_match, best_match_id))

Best_match : 55.06359959174482, Best_match_id : 737


In [35]:
# 코사인 적용
my_id, my_vector = 0, adj_matrix[0] # my_vector는 인접행렬의 0번째
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix) :
    if my_id != user_id : # 다른 id끼리 비교
        cos_similarity = compute_cos_similarity(my_vector, user_vector) # 코사인 공식 함수 적용
        if cos_similarity > best_match : # 코사인 유사도가 더 크면 교체
            best_match        = cos_similarity
            best_match_id     = user_id
            best_match_vector = user_vector

print('Best_match : {}, Best_match_id : {}'.format(best_match, best_match_id))

Best_match : 0.569065731527988, Best_match_id : 915


## 협업 필터링(Collaborative Filtering)

* 사용자와 항목의 유사성을 동시에 고려해 추천
* 기존에 내 관심사가 아닌 항목이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있다.
  * 장점
    * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요 없다.
    * 기존의 관심사가 아니더라도 추천 가능
  * 단점
    * 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
    * 추가 특성을 사용하기 어려움

In [36]:
from surprise import KNNBasic, SVDpp, NMF

In [37]:
data = Dataset.load_builtin('ml-100k', prompt=False)

* KNN을 사용한 협업 필터링

In [38]:
model = KNNBasic()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9815  0.9795  0.9810  0.9750  0.9801  0.9794  0.0023  
MAE (testset)     0.7743  0.7719  0.7767  0.7710  0.7721  0.7732  0.0020  
Fit time          0.27    0.49    0.45    0.55    0.32    0.42    0.10    
Test time         5.76    6.86    7.30    5.78    3.88    5.92    1.18    


{'fit_time': (0.2704000473022461,
  0.48667383193969727,
  0.4485161304473877,
  0.5497815608978271,
  0.3233044147491455),
 'test_mae': array([0.77430988, 0.77194244, 0.77668909, 0.77102481, 0.77213975]),
 'test_rmse': array([0.98145494, 0.9794597 , 0.98104497, 0.97502815, 0.9800566 ]),
 'test_time': (5.763960361480713,
  6.85634183883667,
  7.303537130355835,
  5.775442600250244,
  3.8812220096588135)}

* SVD를 사용한 협업 필터링

In [39]:
model = SVD()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9262  0.9421  0.9371  0.9392  0.9350  0.9359  0.0054  
MAE (testset)     0.7289  0.7414  0.7364  0.7435  0.7377  0.7376  0.0050  
Fit time          10.08   13.03   12.62   10.45   7.13    10.66   2.11    
Test time         0.61    0.49    0.37    0.22    0.14    0.37    0.17    


{'fit_time': (10.0796377658844,
  13.03048586845398,
  12.615183591842651,
  10.452624082565308,
  7.1331446170806885),
 'test_mae': array([0.72886698, 0.74141165, 0.73644739, 0.74350985, 0.73765037]),
 'test_rmse': array([0.9261512 , 0.94210192, 0.93706562, 0.93916388, 0.93500527]),
 'test_time': (0.6103799343109131,
  0.4933631420135498,
  0.36926817893981934,
  0.21935749053955078,
  0.13737702369689941)}

* NMF를 사용한 협업 필터링

In [40]:
model = NMF()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9676  0.9752  0.9690  0.9519  0.9513  0.9630  0.0097  
MAE (testset)     0.7619  0.7661  0.7608  0.7482  0.7482  0.7571  0.0074  
Fit time          10.27   11.91   13.27   11.04   7.58    10.81   1.90    
Test time         0.44    0.60    0.39    0.28    0.11    0.36    0.16    


{'fit_time': (10.266653299331665,
  11.91204309463501,
  13.268490076065063,
  11.040579080581665,
  7.582773923873901),
 'test_mae': array([0.76194778, 0.76613076, 0.76075153, 0.74823444, 0.74822688]),
 'test_rmse': array([0.96761604, 0.97519751, 0.9689882 , 0.95186688, 0.95131721]),
 'test_time': (0.43580150604248047,
  0.6024918556213379,
  0.39461779594421387,
  0.2812502384185791,
  0.11064410209655762)}

* SVDpp를 사용한 협업 필터링

In [ ]:
model = SVDpp()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

## 하이브리드(Hybrid)

* 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
* 많은 하이브리드 방식이 존재
* 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

In [58]:
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import Dataset

In [59]:
# load data
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)

# 0부터 시작
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [60]:
n_users  = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])
shape    = (n_users + 1, n_movies + 1)
shape

(943, 1682)

In [63]:
adj_matrix = np.ndarray(shape, dtype=int)

for user_id, movie_id, rating, time in raw_data :
    adj_matrix[user_id][movie_id] = rating

adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [64]:
# U : 사용자, S : 특이값 벡터, V : 항목(아이템)
U, S, V = randomized_svd(adj_matrix, n_components=2) # 2차원으로 만듦
S = np.diag(S)

In [65]:
print(U.shape)
print(S.shape)
print(V.shape)

(943, 2)
(2, 2)
(2, 1682)


In [66]:
np.matmul(np.matmul(U, S), V)

array([[ 3.91732663e+00,  1.47276644e+00,  7.98261988e-01, ...,
         6.24907189e-04,  1.41100852e-02,  1.36545878e-02],
       [ 1.85777226e+00,  3.96191175e-01,  5.05705740e-01, ...,
         5.38862978e-03,  1.77237914e-03,  5.26968095e-04],
       [ 8.94989517e-01,  1.71578497e-01,  2.51738682e-01, ...,
         2.92094923e-03,  5.39937171e-04, -1.25733753e-04],
       ...,
       [ 9.92051955e-01,  2.10814957e-01,  2.70363365e-01, ...,
         2.89019297e-03,  9.34221962e-04,  2.66612193e-04],
       [ 1.30425401e+00,  5.27669941e-01,  2.50080165e-01, ...,
        -4.20677765e-04,  5.30525683e-03,  5.28069948e-03],
       [ 2.82999397e+00,  9.70812247e-01,  6.15871694e-01, ...,
         2.02091492e-03,  8.67740813e-03,  8.03107892e-03]])

* 사용자 기반 추천
* 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
* 사용자 특징 벡터의 유사도 사용

In [67]:
# 코사인 적용
my_id, my_vector = 0, U[0] # my_vector는 인접행렬의 0번째
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U) :
    if my_id != user_id : # 다른 id끼리 비교
        cos_similarity = compute_cos_similarity(my_vector, user_vector) # 코사인 공식 함수 적용
        if cos_similarity > best_match : # 코사인 유사도가 더 크면 교체
            best_match        = cos_similarity
            best_match_id     = user_id
            best_match_vector = user_vector

print('Best_match : {}, Best_match_id : {}'.format(best_match, best_match_id))

Best_match : 0.9999942295956324, Best_match_id : 235


In [68]:
# 추천리스트
recommend_list = []

for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])) :
    log1, log2 = log
    if log1 < 1 and log2 > 0 : 
        recommend_list.append(i)
    
print(recommend_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


* 항목 기반 추천
* 내가 본 항목과 비슷한 항목을 추천
* 항목 특징 벡터의 유사도 사용

In [69]:
# 코사인 적용
my_id, my_vector = 0, V.T[0] # my_vector는 인접행렬의 0번째
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(V.T) :
    if my_id != user_id : # 다른 id끼리 비교
        cos_similarity = compute_cos_similarity(my_vector, user_vector) # 코사인 공식 함수 적용
        if cos_similarity > best_match : # 코사인 유사도가 더 크면 교체
            best_match        = cos_similarity
            best_match_id     = user_id
            best_match_vector = user_vector

print('Best_match : {}, Best_match_id : {}'.format(best_match, best_match_id))

Best_match : 0.9999999951364145, Best_match_id : 1287


In [70]:
# 추천리스트
recommend_list = []

for i, user_vector in enumerate(adj_matrix) :
    if adj_matrix[i][my_id] > 0.9 :
        recommend_list.append(i)
    
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 

* 비음수 행렬 분해를 사용한 하이브리드 추천

In [71]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [72]:
A, B, iter = non_negative_factorization(adj_matrix, n_components=2)

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1035: FutureWarning: The default value of init will change from random to None in 0.23 to make it consistent with decomposition.NMF.
  "with decomposition.NMF.", FutureWarning)


In [73]:
np.matmul(A, B)

array([[3.71104018e+00, 1.48480219e+00, 7.39556125e-01, ...,
        3.64528320e-03, 1.45530916e-02, 1.44128805e-02],
       [2.11742101e+00, 2.36688755e-01, 5.51606634e-01, ...,
        4.76761892e-03, 2.39680318e-05, 0.00000000e+00],
       [9.85450047e-01, 1.10155205e-01, 2.56718329e-01, ...,
        2.21885505e-03, 1.11547482e-05, 0.00000000e+00],
       ...,
       [1.04460934e+00, 1.16768126e-01, 2.72129840e-01, ...,
        2.35205905e-03, 1.18243985e-05, 0.00000000e+00],
       [1.45794160e+00, 5.42097714e-01, 2.99301518e-01, ...,
        1.61362511e-03, 5.15824292e-03, 5.10694298e-03],
       [2.44655214e+00, 9.41444629e-01, 4.95510295e-01, ...,
        2.56798945e-03, 9.08667872e-03, 8.99767821e-03]])

* 사용자 기반 추천

In [76]:
# 코사인 적용
my_id, my_vector = 0, A[0] # my_vector는 인접행렬의 0번째
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(A) :
    if my_id != user_id : # 다른 id끼리 비교
        cos_similarity = compute_cos_similarity(my_vector, user_vector) # 코사인 공식 함수 적용
        if cos_similarity > best_match : # 코사인 유사도가 더 크면 교체
            best_match        = cos_similarity
            best_match_id     = user_id
            best_match_vector = user_vector

print('Best_match : {}, Best_match_id : {}'.format(best_match, best_match_id))

Best_match : 0.9999999801565269, Best_match_id : 314


In [77]:
# 추천리스트
recommend_list = []

for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])) :
    log1, log2 = log
    if log1 < 1 and log2 > 0 : 
        recommend_list.append(i)
    
print(recommend_list)

[272, 275, 284, 285, 287, 300, 301, 302, 304, 317, 323, 326, 339, 381, 427, 430, 432, 460, 465, 474, 503, 507, 512, 519, 522, 530, 602, 641, 644, 650, 653, 656, 672, 708, 731, 740, 745, 769, 791, 1064, 1083]


* 항목 기반 추천

In [78]:
my_id, my_vector = 0, B.T[0]  # 기준 항목이 0번 이라고 가정
best_match, best_match_id, best_match_vector = -1, -1, []

for item_id, item_vector in enumerate(V.T):
  if my_id != item_id:
    cos_similarity = compute_cos_similarity(my_vector, item_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = item_id
      best_match_vector = item_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999999997511131, Best Match ID: 652


In [79]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
  if adj_matrix[i][my_id] > 0.9:
    recommend_list.append(i)
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 